In this notebook, we will demonstrate the concept of ensemble learning. With ensemble models, we combine multiple models together to create a more accurate prediction.

In [27]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# Load the dataset
titanic = sns.load_dataset('titanic')
X, y = titanic.drop(columns='survived'), titanic.survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)





In [28]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [29]:
# Pipeline: Integrate preprocessing into the training process
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier())
])

# Number of bootstrap samples and trees in the ensemble
n_samples = len(X_train)
n_trees = 50

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), size=len(X), replace=True)
    return X[indices], y[indices]



In [30]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [31]:
# Train multiple decision trees on different bootstrap samples
trees = []
for _ in range(n_trees):
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    
    # Clone the pipeline for each tree
    cloned_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
    cloned_pipeline.fit(X_sample, y_sample)
    trees.append(cloned_pipeline)



KeyError: "None of [Index([187, 608, 133, 575, 350, 499, 523, 107, 189, 120,\n       ...\n       238, 519,  97, 571, 240, 233, 525, 104,  80, 447],\n      dtype='int32', length=623)] are in the [columns]"

In [ ]:
trees

[Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassif

In [ ]:
# Aggregate predictions
predictions = np.zeros((len(X_test), n_trees))
for tree_index, tree in enumerate(trees):
    predictions[:, tree_index] = tree.predict(X_test)



In [ ]:
predictions

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Majority vote
y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions.astype('int'))



In [ ]:
# Test accuracy
print(f"Accuracy (Custom Bagged Trees): {accuracy_score(y_test, y_pred)*100:.2f}%")

Accuracy (Custom Bagged Trees): 100.00%
